In [133]:
import requests as re
from bs4 import BeautifulSoup as bs
import pandas as pd

In [76]:
url = 'https://www.baseball-reference.com'

In [13]:
page_html = re.get(url)
soup = bs(page_html.text)

In [29]:
teams = soup.find_all('div',class_='division')

In [78]:
year = '2021'
new_list = []
for division in teams:
    team_names = division.find_all('a')
    for links in team_names:
        final_link = links['href'].replace('/teams','')
        final_link = url + '/teams/'+ final_link + year + '.shtml'
        new_list.append(final_link)

In [45]:
new_list

['https://www.baseball-reference.com/teams/ATL/2021.shtml',
 'https://www.baseball-reference.com/teams/FLA/2021.shtml',
 'https://www.baseball-reference.com/teams/NYM/2021.shtml',
 'https://www.baseball-reference.com/teams/PHI/2021.shtml',
 'https://www.baseball-reference.com/teams/WSN/2021.shtml',
 'https://www.baseball-reference.com/teams/CHC/2021.shtml',
 'https://www.baseball-reference.com/teams/CIN/2021.shtml',
 'https://www.baseball-reference.com/teams/MIL/2021.shtml',
 'https://www.baseball-reference.com/teams/PIT/2021.shtml',
 'https://www.baseball-reference.com/teams/STL/2021.shtml',
 'https://www.baseball-reference.com/teams/ARI/2021.shtml',
 'https://www.baseball-reference.com/teams/COL/2021.shtml',
 'https://www.baseball-reference.com/teams/LAD/2021.shtml',
 'https://www.baseball-reference.com/teams/SDP/2021.shtml',
 'https://www.baseball-reference.com/teams/SFG/2021.shtml',
 'https://www.baseball-reference.com/teams/BAL/2021.shtml',
 'https://www.baseball-reference.com/tea

In [81]:
def player_links_from_team(team_url):
    raw_data = re.get(team_url)
    html_of_team_page = bs(raw_data.text)
    batting_table = html_of_team_page.find('div',id='div_team_batting')
    all_players = batting_table.find('tbody').find_all('tr')
    list_of_player_links = []
    for i in all_players:
        position = i.find(attrs={'data-stat':'pos'}).text
        if position != 'P' and position != 'Pos':
            player_link = i.find('a')['href']
            list_of_player_links.append(url+player_link)
    return list_of_player_links

In [86]:
list_of_players = player_links_from_team(new_list[0])

In [540]:
#make dictionary for each line and then add each line to a list
def data_pull_player_link(player_link):
    raw_data = re.get(player_link)
    html_of_player_page = bs(raw_data.text)  
    standard_batting = html_of_player_page.find('div',id='div_batting_standard')
    html_of_player_page_string = str(html_of_player_page)
    removing_comments = html_of_player_page_string.replace('<!--','').replace('-->','')
    html_without_comments = bs(removing_comments)
    player_value_batting = html_without_comments.find('div',id='div_batting_value').find('tbody').find_all('tr',class_=['full','full rowSum'])
    try:
        advanced_batting = html_without_comments.find('div',id='div_batting_advanced').find('tbody').find_all('tr',class_=['partial_table','full'])
    except:
        advanced_batting = []
    standard_batting_lines = standard_batting.find('tbody').find_all('tr',class_=['partial_table','full'])
    list_of_years = []
    new_standard = []
    new_advanced = []
    for i in standard_batting_lines:
        if i.find('td',attrs={'data-stat':'team_ID'}).text != 'TOT' and i.find('td',attrs={'data-stat':'team_ID'}).text != '':
            new_standard.append(i)
    for i in advanced_batting:
        if i.find('td',attrs={'data-stat':'team_ID'}).text != 'TOT'and i.find('td',attrs={'data-stat':'team_ID'}).text != '':
            new_advanced.append(i)
    for i,j,k in zip(new_standard,player_value_batting,new_advanced):
        #this filters out the TOTAL rows
        if i.find('td',attrs={'data-stat':'team_ID'}).text != 'TOT':
            year_dict = {}
            year_dict['year'] = i.find('th',attrs={'data-stat':'year_ID'}).text
            year_dict['age'] = i.find('td',attrs={'data-stat':'age'}).text
            year_dict['team_ID'] = i.find('td',attrs={'data-stat':'team_ID'}).text
            year_dict['games'] = i.find('td',attrs={'data-stat':'G'}).text
            year_dict['PA'] = i.find('td',attrs={'data-stat':'PA'}).text
            year_dict['AB'] = i.find('td',attrs={'data-stat':'AB'}).text
            year_dict['R'] = i.find('td',attrs={'data-stat':'R'}).text
            year_dict['H'] = i.find('td',attrs={'data-stat':'H'}).text
            year_dict['2B'] = i.find('td',attrs={'data-stat':'2B'}).text
            year_dict['3B'] = i.find('td',attrs={'data-stat':'3B'}).text
            year_dict['HR'] = i.find('td',attrs={'data-stat':'HR'}).text
            year_dict['RBI'] = i.find('td',attrs={'data-stat':'RBI'}).text
            year_dict['SB'] = i.find('td',attrs={'data-stat':'SB'}).text
            year_dict['CS'] = i.find('td',attrs={'data-stat':'CS'}).text
            year_dict['BB'] = i.find('td',attrs={'data-stat':'BB'}).text
            year_dict['SO'] = i.find('td',attrs={'data-stat':'SO'}).text
            year_dict['batting_avg'] = i.find('td',attrs={'data-stat':'batting_avg'}).text
            year_dict['onbase_perc'] = i.find('td',attrs={'data-stat':'onbase_perc'}).text
            year_dict['slugging_perc'] = i.find('td',attrs={'data-stat':'slugging_perc'}).text
            year_dict['onbase_plus_slugging'] = i.find('td',attrs={'data-stat':'onbase_plus_slugging'}).text
            year_dict['onbase_plus_slugging_plus'] = i.find('td',attrs={'data-stat':'onbase_plus_slugging_plus'}).text
            year_dict['TB'] = i.find('td',attrs={'data-stat':'TB'}).text
            year_dict['GIDP'] = i.find('td',attrs={'data-stat':'GIDP'}).text
            year_dict['HBP'] = i.find('td',attrs={'data-stat':'HBP'}).text
            year_dict['SH'] = i.find('td',attrs={'data-stat':'SH'}).text
            year_dict['SF'] = i.find('td',attrs={'data-stat':'SF'}).text
            year_dict['IBB'] = i.find('td',attrs={'data-stat':'IBB'}).text
            year_dict['Pos'] = i.find('td',attrs={'data-stat':'pos_season'}).text
            year_dict['awards'] = i.find('td',attrs={'data-stat':'award_summary'}).text
            year_dict['runs_bat'] = j.find('td',attrs={'data-stat':'runs_bat'}).text
            year_dict['runs_br'] = j.find('td',attrs={'data-stat':'runs_br'}).text
            year_dict['runs_dp'] = j.find('td',attrs={'data-stat':'runs_dp'}).text
            year_dict['runs_fielding'] = j.find('td',attrs={'data-stat':'runs_fielding'}).text
            year_dict['runs_above_avg_bat'] = j.find('td',attrs={'data-stat':'runs_above_avg_bat'}).text
            year_dict['WAA'] = j.find('td',attrs={'data-stat':'WAA'}).text
            year_dict['runs_replacement'] = j.find('td',attrs={'data-stat':'runs_replacement'}).text
            year_dict['runs_above_rep'] = j.find('td',attrs={'data-stat':'runs_above_rep'}).text
            year_dict['WAR'] = j.find('td',attrs={'data-stat':'WAR'}).text
            year_dict['waa_win_perc'] = j.find('td',attrs={'data-stat':'waa_win_perc'}).text
            year_dict['waa_win_perc_162'] = j.find('td',attrs={'data-stat':'waa_win_perc_162'}).text
            year_dict['WAR_off'] = j.find('td',attrs={'data-stat':'WAR_off'}).text
            year_dict['WAR_def'] = j.find('td',attrs={'data-stat':'WAR_def'}).text
            year_dict['runs_above_rep_off'] = j.find('td',attrs={'data-stat':'runs_above_rep_off'}).text
            year_dict['Salary'] = j.find('td',attrs={'data-stat':'Salary'}).text
            year_dict['rOBA'] = k.find('td',attrs={'data-stat':'rOBA'}).text
            year_dict['Rbat+'] = k.find('td',attrs={'data-stat':'Rbat+'}).text
            year_dict['adv_bat_babip'] = k.find('td',attrs={'data-stat':'adv_bat_babip'}).text
            year_dict['adv_bat_iso'] = k.find('td',attrs={'data-stat':'adv_bat_iso'}).text
            year_dict['adv_bat_home_run_perc'] = k.find('td',attrs={'data-stat':'adv_bat_home_run_perc'}).text
            year_dict['adv_bat_strikeout_perc'] = k.find('td',attrs={'data-stat':'adv_bat_strikeout_perc'}).text
            year_dict['adv_bat_base_on_balls_perc'] = k.find('td',attrs={'data-stat':'adv_bat_base_on_balls_perc'}).text
            year_dict['adv_bat_exit_velo'] = k.find('td',attrs={'data-stat':'adv_bat_exit_velo'}).text
            year_dict['adv_bat_hard_hit_perc'] = k.find('td',attrs={'data-stat':'adv_bat_hard_hit_perc'}).text
            year_dict['adv_bat_ld_perc'] = k.find('td',attrs={'data-stat':'adv_bat_ld_perc'}).text
            year_dict['adv_bat_gb_perc'] = k.find('td',attrs={'data-stat':'adv_bat_gb_perc'}).text
            year_dict['adv_bat_fb_perc'] = k.find('td',attrs={'data-stat':'adv_bat_fb_perc'}).text
            year_dict['adv_bat_gb_fb_ratio'] = k.find('td',attrs={'data-stat':'adv_bat_gb_fb_ratio'}).text
            year_dict['adv_bat_pull_perc'] = k.find('td',attrs={'data-stat':'adv_bat_pull_perc'}).text
            year_dict['adv_bat_cent_perc'] = k.find('td',attrs={'data-stat':'adv_bat_cent_perc'}).text
            year_dict['adv_bat_oppo_perc'] = k.find('td',attrs={'data-stat':'adv_bat_oppo_perc'}).text
            year_dict['adv_bat_wpa_bat'] = k.find('td',attrs={'data-stat':'adv_bat_wpa_bat'}).text
            year_dict['adv_bat_cwpa_bat'] = k.find('td',attrs={'data-stat':'adv_bat_cwpa_bat'}).text
            year_dict['adv_bat_re24_bat'] = k.find('td',attrs={'data-stat':'adv_bat_re24_bat'}).text
            list_of_years.append(year_dict)              
    return list_of_years
    

In [467]:
list_of_players[-1]

'https://www.baseball-reference.com/players/k/kazmase01.shtml'

In [468]:
test3 = data_pull_player_link(list_of_players[-1])

In [469]:
test3 = pd.DataFrame(test3)

In [470]:
test3

,year,age,team_ID,games,PA,AB,R,H,2B,3B,...,adv_bat_ld_perc,adv_bat_gb_perc,adv_bat_fb_perc,adv_bat_gb_fb_ratio,adv_bat_pull_perc,adv_bat_cent_perc,adv_bat_oppo_perc,adv_bat_wpa_bat,adv_bat_cwpa_bat,adv_bat_re24_bat
0,2008,23,SDP,19,46,39,2,8,1,0,...,18.5%,40.7%,14.8%,0.85,25.9%,59.3%,14.8%,-0.6,0.0%,-3.0
1,2021,36,ATL,3,2,2,0,0,0,0,...,0.0%,100.0%,0.0%,,50.0%,50.0%,0.0%,0.0,0.0%,-0.7


In [485]:
def list_of_players_to_df(list_of_players):
    final_dataframe = []
    for i in list_of_players:
        stats_of_player = data_pull_player_link(i)
        for j in stats_of_player:
            final_dataframe.append(j)
    return final_dataframe

In [472]:
braves = list_of_players_to_df(list_of_players)

In [473]:
braves = pd.DataFrame(braves)

In [474]:
braves

,year,age,team_ID,games,PA,AB,R,H,2B,3B,...,adv_bat_ld_perc,adv_bat_gb_perc,adv_bat_fb_perc,adv_bat_gb_fb_ratio,adv_bat_pull_perc,adv_bat_cent_perc,adv_bat_oppo_perc,adv_bat_wpa_bat,adv_bat_cwpa_bat,adv_bat_re24_bat
0,2013,24,NYM,31,112,99,4,20,3,0,...,19.0%,48.1%,26.6%,0.93,19.0%,67.1%,13.9%,-0.2,0.0%,-5.8
1,2014,25,NYM,108,421,385,48,93,22,3,...,24.8%,42.4%,25.1%,0.74,24.6%,61.4%,14.0%,-0.4,-0.9%,0.3
2,2015,26,NYM,67,268,239,31,64,14,1,...,24.5%,37.5%,25.5%,0.60,26.6%,48.4%,25.0%,1.2,0.9%,9.6
3,2016,27,NYM,75,276,251,27,62,7,0,...,21.1%,51.5%,17.6%,1.09,24.0%,60.8%,15.2%,-1.4,-0.9%,-11.1
4,2017,28,NYM,112,376,348,39,85,19,1,...,23.6%,42.5%,24.3%,0.74,22.9%,55.1%,21.9%,-0.3,-0.1%,-1.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2019,36,TEX,86,244,228,17,36,9,0,...,23.9%,40.1%,23.2%,0.69,20.4%,59.2%,20.4%,-2.3,-0.8%,-25.1
205,2020,37,TEX,24,68,62,6,10,1,1,...,33.3%,33.3%,17.9%,0.52,33.3%,56.4%,10.3%,-0.3,0.0%,-3.7
206,2021,38,ATL,3,9,9,0,0,0,0,...,25.0%,25.0%,25.0%,0.50,25.0%,25.0%,50.0%,-0.2,-0.1%,-2.1
207,2008,23,SDP,19,46,39,2,8,1,0,...,18.5%,40.7%,14.8%,0.85,25.9%,59.3%,14.8%,-0.6,0.0%,-3.0


In [513]:
def final_dataframe_builder(list_of_teams):
    final_dataframe = []
    for i in list_of_teams:
        list_of_players_ = player_links_from_team(i)
        answer = list_of_players_to_df(list_of_players_)
        for k in answer:
            final_dataframe.append(k)
    return final_dataframe

In [523]:
nl_east = new_list[0:5]
nl_central = new_list[5:10]
nl_west = new_list[10:15]
al_east = new_list[15:20]
al_central = new_list[20:25]
al_west = new_list[25:30]

In [522]:
nl_east_df = pd.DataFrame(final_dataframe_builder(nl_east))

In [524]:
nl_central_df = pd.DataFrame(final_dataframe_builder(nl_central))

In [525]:
nl_west_df = pd.DataFrame(final_dataframe_builder(nl_west))

In [528]:
al_east_df = pd.DataFrame(final_dataframe_builder(al_east))

In [529]:
al_central_df = pd.DataFrame(final_dataframe_builder(al_central))

In [541]:
al_west_df = pd.DataFrame(final_dataframe_builder(al_west))

In [543]:
final_df = pd.concat([nl_east_df,nl_central_df,nl_west_df,al_east_df,al_central_df,al_west_df])

In [546]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4163 entries, 0 to 669
Data columns (total 63 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   year                        4163 non-null   object
 1   age                         4163 non-null   object
 2   team_ID                     4163 non-null   object
 3   games                       4163 non-null   object
 4   PA                          4163 non-null   object
 5   AB                          4163 non-null   object
 6   R                           4163 non-null   object
 7   H                           4163 non-null   object
 8   2B                          4163 non-null   object
 9   3B                          4163 non-null   object
 10  HR                          4163 non-null   object
 11  RBI                         4163 non-null   object
 12  SB                          4163 non-null   object
 13  CS                          4163 non-null   objec

In [547]:
final_df.to_csv('baseball_players.csv')